# Test case: Glucosinolates

For this example, we will use the model "PlantCoreMetabolism_v2_0_0.xml"
derived from the article *Flux balance analysis of metabolism during growth by
osmotic cell expansion and its application to tomato fruits*. The model got its
metabolites' protonation states removed.

-----
**GOAL:** We will calculate the metabolic cost for synthesizing three specialized
compounds called *glucosinolates*, which give the typical taste for plants from
the Brassicacea family.

The glucosinolates to insert are:  

| Trivial name | Compound name | Biocyc identifier |
|-|-|-|
| 3-methylsulfinylpropyl | glucoiberin | 3-METHYLSULFINYLPROPYL-GLUCOSINOLATE |
| 4-methylthiobutyl | glucoerucin | CPDQT-281 |
| 4-methylsulfinylbutyl | glucoraphanin | CPDQT-280 |

Glucoerucin and glucoraphanin are derived from dihomomethionine, while glucoiberin
is obtained from homomethionine

------

First, we will load the function read_sbml_model from COBRApy and the native
filesystem path module from python. Then, we will load two functions from
CobraMod, add_pathway and add_reactions

In [1]:
from pathlib import Path

from cobra.io import read_sbml_model

from cobramod import add_pathway, add_reactions, __version__

In [2]:
print(__version__)

0.4.2


We will load the model and save it as `main_model`. However, to avoid overwriting
the main model, we will use a copy, which is going to be used throughout the
whole file. Additionally, we will define the path for the data and the metabolites
that we want to ignore when CobraMod performs the non-zero flux test

In [3]:
# this object defines the directory for the data
dir_data = Path.cwd().joinpath("data")

main_model = read_sbml_model(
    str(dir_data.joinpath("PlantCoreMetabolism_v2_0_0_deprotonated.sbml"))
)

# These metabolites will not be used in the non-zero flux test
ignore_list = [
    "WATER_c", "PROTON_c", "OXYGEN_MOLECULE_c", "UDP_c",
    "Pi_p", "ADP_p"
]

The next step is to define our enviroment for our model. Since we are working
with a generic model of a plant, it is wise to constrain reactions that would
prevent an autotrophic enviroment. In our case we have three boundary reactions to modify:  

`Sucrose_tx`, `GLC_tx`: Responsible for sugar intake  
`Photon_tx`: Amount of light absorbed

In [4]:
model = main_model.copy()

# Defining objective function
model.objective = "AraCore_Biomass_tx"

# Constraining enviroment (autotroph)
model.exchanges.Sucrose_tx.bounds = (0,0)
model.exchanges.GLC_tx.bounds = (0,0)
model.exchanges.Photon_tx.bounds = (0,250)

# Saving original solution
solution_original = model.optimize()
model.summary()

## Precursors

To include some glucosinolates, we not only need to include their pathways, but
also to include the synthesis of some metabolites that are used for their
biosynthesis. In this scenario, we need to add the following metabolites.

- Glutathione
- 3'-phosphoadenylyl-sulfate
- Adenosine 3',5'-bisphosphate

Additionally, we need to add homomethionine and dihomomethionine in order to
synthesize the glucosinolates. If these metabolites are not included,
CobraMod will create boundary reactions (Sink reactions) for them.
If this happens, then some metabolic costs will be neglected.

There are multiple way to extend our model with these precursors. To begin with,
we will use the function `add_reactions` and pass the path of a file with custom
transporter reaction that we need.

In [5]:
!cat reactions.txt

add_reactions(
    model=model,
    # Using a Path object
    obj=dir_data.joinpath("reactions_GLS.txt"),
    database="ARA",
    directory=dir_data,
)

cat: reactions.txt: No such file or directory


cobramod/utils.py:53 UserWarning: Reaction "NADPH_FERRIHEMOPROTEIN_REDUCTASE_RXN_rc" unbalanced. Following atoms are affected. Please verify:
{'H': -2.0}


### Glutathione

The glucosinolate-precursor is always converted to a Glutathione-S-conjugate,
which can be later hydrolyzed to continue with the synthesis of the glucosinolate
Both reactions for this [pathway
](https://biocyc.org/META/new-image?object=GLUTATHIONESYN-PWY) are located in
the chloroplast. We will use the function `add_pathway` to include the pathway.

In [6]:
add_pathway(
    model=model,
    # Using the identifier
    pathway="GLUTATHIONESYN-PWY",
    # Our model defines 'p' as chloroplast
    compartment="p",
    database="ARA",
    directory=dir_data,
    ignore_list=ignore_list,
)
# We can see at the end of the table, the new reactions
model.optimize()

Changes:
Reactions 	2
Metabolites	1
Exchange 	0
Demand		0
Sinks		0
Genes		2
Groups		1



,fluxes,reduced_costs
PRO_PROTON_vc,0.000000e+00,-6.938894e-18
Ca_tx,2.729743e-01,0.000000e+00
H2O_xc,7.382353e-16,-0.000000e+00
sCIT_biomass,-8.180567e-02,1.387779e-17
ACETYLGLUTKIN_RXN_p,7.714903e-02,-1.474515e-17
...,...,...
PAPS_pc,0.000000e+00,0.000000e+00
HOMOMETHIONINE_pc,0.000000e+00,0.000000e+00
CPDQT_340_pc,0.000000e+00,0.000000e+00
GLUTCYSLIG_RXN_p,0.000000e+00,-2.350888e-01


We can call the method `visualize` to inspect the reactions for
this pathway.

In [7]:
model.groups.get_by_id("GLUTATHIONESYN-PWY").visualize()

Builder(reaction_styles=['text'])

We can see that CobraMod was able to insert these new reactions into
our model. With help of Escher we can identify which metabolites are
used in the reactions.

### 3'-phosphoadenylyl-sulfate

In Biocyc, its identifier is PAPS. This compound is used by the enzyme
desulfoglucosinolate sulfotransferase to transform the precursors into
glucosinolates

In [8]:
add_pathway(
    model=model,
    pathway="PWY-5340",
    compartment="p",
    database="ARA",
    directory=dir_data,
    ignore_list=ignore_list,
)
model.optimize()

Changes:
Reactions 	1
Metabolites	0
Exchange 	0
Demand		0
Sinks		0
Genes		4
Groups		1



,fluxes,reduced_costs
PRO_PROTON_vc,0.000000,-6.938894e-18
Ca_tx,0.272974,0.000000e+00
H2O_xc,0.000000,-0.000000e+00
sCIT_biomass,-0.081806,-0.000000e+00
ACETYLGLUTKIN_RXN_p,0.077149,3.469447e-18
...,...,...
HOMOMETHIONINE_pc,0.000000,0.000000e+00
CPDQT_340_pc,0.000000,0.000000e+00
GLUTCYSLIG_RXN_p,0.000000,-2.350888e-01
GLUTATHIONE_SYN_RXN_p,0.000000,0.000000e+00


### Adenosine 3',5'-bisphosphate

This compound is a product from the sulfotransferase. Currently, our model
does not contain reactions that can reutilize it. Thus, we will include a
reaction that can hydrolyze it.

In [9]:
add_reactions(
    model=model,
    # This function can use single strings
    obj="325-BISPHOSPHATE-NUCLEOTIDASE-RXN, c",
    database="ARA",
    directory=dir_data,
)

### Homomethionine

The first part of this pathway, i.e. transmination of methionine,
occurs in the cytosol. The rest occurs in the chloroplast including its elongation
for the synthesis of other glucosinolate precursors such as dihomomethionine.

To simulate the different compartments, we will add the first reaction to the cytosol.
Then, we need to create a transport reaction to continue with the flow of the pathway:

In [10]:
add_reactions(
    model=model,
    obj=[
        # First reaction of the pathway
        "R15-RXN, c",
        # This is the transport to chloroplast
        "CPD_479_cp, CPD_479_cp| CPD_479_c <-> CPD_479_p"
    ],
    database="ARA",
    directory=dir_data,
    # We have to use non-generic metabolites
    replacement={
        "Amino-Acids-20": "GLT",
        "2-Oxo-carboxylates": "2-KETOGLUTARATE",
    }
)

# Continue with the rest of the pathway
add_pathway(
    model=model,
    pathway="PWY-1186",
    # The pathway is found in the chloroplast
    compartment="p",
    database="ARA",
    directory=dir_data,
    ignore_list=ignore_list,
    replacement={
        "Amino-Acids-20":"GLT",
        "2-Oxo-carboxylates":"2-KETOGLUTARATE",
    },
    # This reaction is added previously but in the cytosol
    avoid_list=["R15-RXN"]
)

model.optimize()

Changes:
Reactions 	6
Metabolites	5
Exchange 	0
Demand		0
Sinks		0
Genes		4
Groups		1



,fluxes,reduced_costs
PRO_PROTON_vc,0.000000e+00,-6.938894e-18
Ca_tx,2.729743e-01,0.000000e+00
H2O_xc,9.289710e-16,-0.000000e+00
sCIT_biomass,-8.180567e-02,-0.000000e+00
ACETYLGLUTKIN_RXN_p,7.714903e-02,-6.071532e-18
...,...,...
RXN_18182_p,0.000000e+00,-0.000000e+00
RXN_18183_p,0.000000e+00,-0.000000e+00
RXN_2204_p,0.000000e+00,0.000000e+00
RXN_18198_p,0.000000e+00,0.000000e+00


In [11]:
model.groups.get_by_id("PWY-1186").visualize()

Builder(reaction_styles=['text'])

### Dihomomethionine

Dihomomethionine is synthesized from the [elongantion chain of homomethionine
](https://metacyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWYQT-4450&detail-level=2).
This pathway is very long and for now, we are only interested in dihomomethionine.
For this reason, we will manually include only the reaction that we are interested,
which are the first 6 reactions.

In [12]:
add_pathway(
    model=model,
    # We can pass a list with reaction identifiers as well
    pathway=[
        "RXNQT-4163", "RXN-18184", "RXN-18185", "RXN-18208", "RXN-18209", "RXNQT-4345"
    ],
    # We can define a custom name for the pathway
    group="PWYQT-4450",
    compartment="p",
    database="ARA",
    directory=dir_data,
    ignore_list=ignore_list,
    replacement={
        "Amino-Acids-20":"GLT",
        "2-Oxo-carboxylates":"2-KETOGLUTARATE",
    },
)

Changes:
Reactions 	6
Metabolites	5
Exchange 	0
Demand		0
Sinks		0
Genes		0
Groups		1



We can even show our pathway vertically

In [13]:
model.groups.get_by_id("PWYQT-4450").visualize(vertical=True)

Builder(reaction_styles=['text'])

## Synthesis from Homomethionine

The first steps for the synthesis of [glucoiberin
](https://biocyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWY-1187&detail-level=2)
are located in the cytosol, though a few hemoproteins are located in the endoplasmatic
reticulum. Hence, we will add custom reactions for the first steps to simulate these
reactions. Then, we will add the rest of the pathway normally with a few extra arguments.

In [14]:
add_reactions(
    model=model,
    obj=[
        # First custom reaction
        "RXN_2206_rc, RXN_2206_rc|"
        +"HOMOMETHIONINE_c + 2 OXYGEN-MOLECULE_c + 2 Red-NADPH-Hemoprotein-Reductases_r -->"
        +"3-METHYLTHIOPROPANALDOXIME_c + CARBON-DIOXIDE_c + 2 Ox-NADPH-Hemoprotein-Reductases_r + 3 WATER_c",

        # Second custom reaction
        "RXN_11414_rc, RXN_11414_rc|"
        +"3-METHYLTHIOPROPANALDOXIME_c + OXYGEN-MOLECULE_c + Red-NADPH-Hemoprotein-Reductases_r <->"
        +"CPD-12389_c + Ox-NADPH-Hemoprotein-Reductases_r +  WATER_c"
    ],
    database="ARA",
    directory=dir_data,   
)

# Continue with rest of the reactions
add_pathway(
    model=model,
    pathway="PWY-1187",
    compartment="c",
    database="ARA",
    directory=dir_data,
    ignore_list=ignore_list,
    avoid_list=[
        # Custom reactions were created already
        "RXN-2206", "RXN-11414",
        # These reactions are not needed since we want one specific compound
        "RXN-11438", "RXN-2222", "RXN-2223", "RXN-2224", "RXN-11445"
    ],
    # We use UDP-GLUCOSE as UDP-α-D-glucose
    replacement = {"CPD-12575": "UDP-GLUCOSE"}
)

cobramod/utils.py:53 UserWarning: Reaction "RXN_2206_rc" unbalanced. Following atoms are affected. Please verify:
{'H': 4.0}
cobramod/utils.py:53 UserWarning: Reaction "RXN_11414_rc" unbalanced. Following atoms are affected. Please verify:
{'H': 2.0}


Changes:
Reactions 	7
Metabolites	7
Exchange 	0
Demand		0
Sinks		1
Genes		12
Groups		1



In [15]:
model.groups.get_by_id("PWY-1187").visualize()

Builder(reaction_styles=['text'])

## Synthesis from Dihomomethionine

As with homomethionine, the first steps of the [glucosinolates synthesis from
dihomomethionine
](https://metacyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWYQT-4471&&detail-level=2)
are located in the cytosol with hemoproteins in the endoplasmatic reticulum.
Thus, we will write custom reactions and then follow the workflow adding a list
of reactions.

In [16]:
add_reactions(
    model=model,
    obj=[
        # First custom reaction
        "RXNQT_4309_rc, RXNQT_4309_rc|"
        +"CPDQT-340_c + 2 OXYGEN-MOLECULE_c + 2 Red-NADPH-Hemoprotein-Reductases_r <->"
        +"CARBON-DIOXIDE_c + CPDQT-341_c + 2 Ox-NADPH-Hemoprotein-Reductases_r + 3 WATER_c",

        # Second custom reaction
        "RXN_11415_rc, RXN_11415_rc|"
        +"CPDQT-341_c + OXYGEN-MOLECULE_c + Red-NADPH-Hemoprotein-Reductases_r <->"
        +"CPD-12392_c + Ox-NADPH-Hemoprotein-Reductases_r + WATER_c"
    ],
    database="ARA",
    directory=dir_data,   
)

# Continue with list
add_pathway(
    model=model,
    # Reactions to synthesize two glucosinolates
    pathway=[
        'RXN-11423','RXN-11431', 'RXN-19588','RXNQT-4319','RXNQT-4324','RXNQT-4329','RXNQT-4334'
    ],
    group="PWYQT-4471",
    compartment="c",
    database="ARA",
    directory=dir_data,
    ignore_list=ignore_list,
    # We use UDP-GLUCOSE as UDP-α-D-glucose
    replacement = {"CPD-12575": "UDP-GLUCOSE"}
)


cobramod/utils.py:53 UserWarning: Reaction "RXNQT_4309_rc" unbalanced. Following atoms are affected. Please verify:
{'H': 4.0}
cobramod/utils.py:53 UserWarning: Reaction "RXN_11415_rc" unbalanced. Following atoms are affected. Please verify:
{'H': 2.0}


Changes:
Reactions 	7
Metabolites	7
Exchange 	0
Demand		0
Sinks		1
Genes		1
Groups		1



In [17]:
model.groups.get_by_id("PWYQT-4471").visualize()

Builder(reaction_styles=['text'])

## Calculating costs

By default, CobraMod will create boundary reactions (in this case, called
sink reactions) during the non-zero test for metabolites that only have
one reaction. Since glucoraphanin and glucoiberin are the end produts of
these pathways, CobraMod built for them extra reactions. On the other hand,
Glucoerucin is a precursor of glucoraphanin, so no boundary reaction was needed.

In [18]:
model.sinks

[<Reaction SK_3_METHYLSULFINYLPROPYL_GLUCOSINOLATE_c at 0x7fd560fc7710>,
 <Reaction SK_CPDQT_280_c at 0x7fd560f24310>]

To calculate te metabolic costs of these 3 glucosinolates, we need to
set the amount of glucosinolates that we require. We use the values according
to [this paper](https://doi.org/10.1016/s0031-9422(02)00549-6). We will use the
data from Table 2, specifically for the rosette leaves at a vegetative stage.
The unit used for the glucosinolates are $\mu mol/gDW$

Using a copy of the model will avoid overwriting the model. Then, we will remove
the extra boundary reactions and define the amount of glucosinolates. Lastly,
a 'for loop' can be use to create demand reactions for these compounds.

In [19]:
model2 = model.copy()
# Removing default sinks
model2.remove_reactions(reactions=model2.sinks)

amount = {
    "3_METHYLSULFINYLPROPYL_GLUCOSINOLATE_c": 1.56 / 1000,
    "CPDQT_280_c": 10.63 / 1000,
    "CPDQT_281_c": 0.84 / 1000
}

# Creating a demand for each glucosinolate with its corresponding quantity
for glucosinolate in amount.keys():
    model2.add_boundary(model2.metabolites.get_by_id(glucosinolate), "demand")
    model2.reactions.get_by_id(f'DM_{glucosinolate}').bounds = (amount[glucosinolate], 1000)

# Defining new solution
solution_glucosinolate = model2.optimize()
model2.summary(solution=solution_glucosinolate, threshold=0.00001)

We can see the summary of the original solution saved as `solution_original`.

In [20]:
model.summary(solution=solution_original, threshold=0.00001)

We can now calculate the $\Delta$ of the objective function and comparate it
to the original value of 0.310198

In [21]:
(solution_original.objective_value - solution_glucosinolate.objective_value)/solution_original.objective_value*100

0.9504861333319434

Synthesizing three glucosinolates in our model results in a growth penalty
of 0.95%. This value is likely to grow the more metabolites incorporated
to the model.